<img src='stats/img/topbar.png' />

## Supervised Learning - Classificação

__CONTEÚDO:__<br>
* [2. Árvore de Decisão](#0.)
* [2.1 Aplicação 1: Credit Dataset](#1.)

<a id="0."></a><br>
### 2. Árvore de Decisão

- Modelo muito popular e atrativo principalmente por ser muito fácil de interpretar.
- A versão de classificação da árvore de decisão busca encontrar o maior decremento na impureza dos nós utilizando a métrica de impureza de Gini:

    ${G(t)} = 1 - {\sum \limits _{j=1} ^{c} p_i^{2} }$ , onde G(_t_) é a medida de impureza do nó _**t**_ e _**p**_ a proporção de observações da classe _**c**_ no nó _**t**_.

- Os dados permitem ao modelo criar nós de decisão binários.

<img src='stats/img/tree_ex.png' />

<a id="2."></a><br>
### 2.1 Aplicação 1: Credit Dataset

__ETAPAS:__<br>
* [1. Preparação do problema](#2.1)
* [2. Sumarizar Dados](#2.2)
* [3. Preparar Dados](#2.3)
* [4. Avaliar algoritmos](#2.4)
* [5. Melhorar Modelo(s)](#2.5)
* [6. Finalizar Modelo(s)](#2.6)

<a id="2.1"></a><br>
#### 1. Preparação do problema
* a) Esclarecer o problema
* b) Carregar libraries
* c) Carregar dataset

- Base de dados que contém diversas características de clientes pelas quais deve-se prever bons e maus pagadores

In [32]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import plot_confusion_matrix, accuracy_score, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import ExtraTreesClassifier
%matplotlib inline

In [11]:
credit = pd.read_csv('data/Credit.csv')

<a id="2.2"></a><br>
#### 2. Sumarizar Dados
* a) Estatística descritiva
* b) Visualização de dados e estrutura
* c) EDA

In [12]:
credit.describe()

,duration,credit_amount,installment_commitment,residence_since,age,existing_credits,num_dependents
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,20.903000,3271.258000,2.973000,2.845000,35.546000,1.407000,1.155000
std,12.058814,2822.736876,1.118715,1.103718,11.375469,0.577654,0.362086
min,4.000000,250.000000,1.000000,1.000000,19.000000,1.000000,1.000000
25%,12.000000,1365.500000,2.000000,2.000000,27.000000,1.000000,1.000000
50%,18.000000,2319.500000,3.000000,3.000000,33.000000,1.000000,1.000000
75%,24.000000,3972.250000,4.000000,4.000000,42.000000,2.000000,1.000000
max,72.000000,18424.000000,4.000000,4.000000,75.000000,4.000000,2.000000


In [13]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   checking_status         1000 non-null   object
 1   duration                1000 non-null   int64 
 2   credit_history          1000 non-null   object
 3   purpose                 1000 non-null   object
 4   credit_amount           1000 non-null   int64 
 5   savings_status          1000 non-null   object
 6   employment              1000 non-null   object
 7   installment_commitment  1000 non-null   int64 
 8   personal_status         1000 non-null   object
 9   other_parties           1000 non-null   object
 10  residence_since         1000 non-null   int64 
 11  property_magnitude      1000 non-null   object
 12  age                     1000 non-null   int64 
 13  other_payment_plans     1000 non-null   object
 14  housing                 1000 non-null   object
 15  exist

In [14]:
credit.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,6,'critical/other existing credit',radio/tv,1169,'no known savings',>=7,4,'male single',none,...,'real estate',67,none,own,2,skilled,1,yes,yes,good
1,0<=X<200,48,'existing paid',radio/tv,5951,<100,1<=X<4,2,'female div/dep/mar',none,...,'real estate',22,none,own,1,skilled,1,none,yes,bad
2,'no checking',12,'critical/other existing credit',education,2096,<100,4<=X<7,2,'male single',none,...,'real estate',49,none,own,1,'unskilled resident',2,none,yes,good
3,<0,42,'existing paid',furniture/equipment,7882,<100,4<=X<7,2,'male single',guarantor,...,'life insurance',45,none,'for free',1,skilled,2,none,yes,good
4,<0,24,'delayed previously','new car',4870,<100,1<=X<4,3,'male single',none,...,'no known property',53,none,'for free',2,skilled,2,none,yes,bad


<a id="2.3"></a><br>
#### 3. Preparar Dados
* a) Limpeza de dados
* b) Seleção de características
* c) Transformação dos dados
* d) Dividir conjunto de dados em partes de treino e teste/validação

In [15]:
X = credit.iloc[:,0:20].values
y = credit.iloc[:,20].values

In [16]:
labelencoder = LabelEncoder()
X[:,0] = labelencoder.fit_transform(X[:,0])
X[:,2] = labelencoder.fit_transform(X[:,2])
X[:, 3] = labelencoder.fit_transform(X[:, 3])
X[:, 5] = labelencoder.fit_transform(X[:, 5])
X[:, 6] = labelencoder.fit_transform(X[:, 6])
X[:, 8] = labelencoder.fit_transform(X[:, 8])
X[:, 9] = labelencoder.fit_transform(X[:, 9])
X[:, 11] = labelencoder.fit_transform(X[:, 11])
X[:, 13] = labelencoder.fit_transform(X[:, 13])
X[:, 14] = labelencoder.fit_transform(X[:, 14])
X[:, 16] = labelencoder.fit_transform(X[:, 16])
X[:, 18] = labelencoder.fit_transform(X[:, 18])
X[:, 19] = labelencoder.fit_transform(X[:, 19])

<a id="2.4"></a><br>
#### 4. Avaliar algoritmos
* a) Treinar algoritmos com o conjunto de treino
* b) Avaliar desempenho no conjunto de teste

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [25]:
treemodel = DecisionTreeClassifier()
treemodel.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [26]:
y_pred =treemodel.predict(X_test)
confusao = confusion_matrix(y_test, y_pred)
confusao

array([[ 42,  44],
       [ 50, 164]], dtype=int64)

In [20]:
plt.figure(figsize=(100, 200))
plot_tree(treemodel, filled=True)
plt.savefig('stats/img/tree_eg.png', dip=1800)
plt.show()

In [27]:
acuracia = accuracy_score(y_test, y_pred)
erro = 1 - acuracia
acuracia, erro

(0.6866666666666666, 0.31333333333333335)

In [29]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         bad       0.46      0.49      0.47        86
        good       0.79      0.77      0.78       214

    accuracy                           0.69       300
   macro avg       0.62      0.63      0.62       300
weighted avg       0.69      0.69      0.69       300



<a id="2.5"></a><br>
#### 5. Melhorar Desempenho
* a) Verificar desempenho do(s) algoritmo(s) com os dados normalizados ou padronizados.
* b) Verificar desempenho do(s) algoritmo(s) em diferentes ajustes de parâmetros da técnica.
* c) Avaliar relevância de atributos x ruídos.

In [34]:
#Seleção de Atributos
forest = ExtraTreesClassifier()
forest.fit(X_train, y_train)
relevancia = forest.feature_importances_
relevancia

array([0.11003284, 0.08537205, 0.06227057, 0.06018365, 0.08506945,
       0.04533678, 0.05068975, 0.05278382, 0.04388042, 0.026155  ,
       0.05167855, 0.04960996, 0.06487447, 0.03667643, 0.0365976 ,
       0.03437747, 0.04169801, 0.02328254, 0.03105607, 0.00837454])

In [49]:
#Selecionados manualmente os 6 atributos mais representativos
X_train_selected = X_train[:,[0, 1, 2, 3, 4]]
X_test_selected = X_test[:,[0, 1, 2, 3, 4]]

In [50]:
tree_aj = DecisionTreeClassifier()
tree_aj.fit(X_train_selected, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [51]:
y_pred = tree_aj.predict(X_test_selected)

In [52]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 39  47]
 [ 39 175]]
              precision    recall  f1-score   support

         bad       0.50      0.45      0.48        86
        good       0.79      0.82      0.80       214

    accuracy                           0.71       300
   macro avg       0.64      0.64      0.64       300
weighted avg       0.71      0.71      0.71       300



In [53]:
acuracia = accuracy_score(y_test, y_pred)
erro = 1 - acuracia
acuracia, erro

(0.7133333333333334, 0.2866666666666666)

<a id="2.6"></a><br>
#### 6. Finalizar Modelo
* a) Selecionar parâmetros ajustados do modelo
* b) Criar modelo standalone com o conjunto completo de treino
* c) Armazenar modelo para uso futuro

In [ ]:
from pickle import dump, load

# Salvar modelo 
filename = 'model.sav'
# dump(knn_cv, open(filename, 'wb'))

# Carregar modelo
# loaded_model = load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)
# print(result)